<a href="https://colab.research.google.com/github/HarrisonWBlack/Machine-Learning-CAP-5610/blob/master/Assignment%202/HW2_model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Five layer, dropout, with batch normalization and data augmentation at 25 Epochs.** 
This model is the best of the five.

In [0]:
import matplotlib.pyplot as plt
import keras
import numpy as np

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential

(training_images, training_labels), (testing_images, testing_labels) = cifar10.load_data()

batch_size = 256
epochs = 25
augment_data = True

training_labels = keras.utils.to_categorical(training_labels)
testing_labels = keras.utils.to_categorical(testing_labels)


In [0]:
# .20 79.13
# .30 79.22 elu - 76
# .35 78.22
dropout_rate = 0.2

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=training_images.shape[1:]))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(dropout_rate))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(dropout_rate))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout_rate))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))


In [49]:
model.compile(loss="categorical_crossentropy",
             optimizer="Adam",
             metrics=["accuracy"])

training_images = training_images.astype("float32") / 255
testing_images = testing_images.astype("float32") / 255

if not augment_data:
  model.fit(training_images, training_labels,
           batch_size=batch_size,
           epochs=epochs,
           validation_data=(testing_images, testing_labels),
           shuffle=True)
  
else:
  datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.,
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode="nearest",
    cval=0.,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0)
  
  datagen.fit(training_images)
  model.fit_generator(datagen.flow(training_images, training_labels,
                                  batch_size=batch_size),
                     epochs=epochs,
                     validation_data=(testing_images, testing_labels),
                     workers=1)
  

Epoch 1/25
196/196 [==============================] - 52s 266ms/step - loss: 1.5643 - acc: 0.4444 - val_loss: 3.4484 - val_acc: 0.1326
Epoch 2/25
196/196 [==============================] - 50s 254ms/step - loss: 1.2427 - acc: 0.5558 - val_loss: 2.8235 - val_acc: 0.2383
Epoch 3/25
196/196 [==============================] - 50s 258ms/step - loss: 1.1143 - acc: 0.6028 - val_loss: 1.3549 - val_acc: 0.5282
Epoch 4/25
196/196 [==============================] - 54s 274ms/step - loss: 1.0319 - acc: 0.6327 - val_loss: 1.1891 - val_acc: 0.5958
Epoch 5/25
196/196 [==============================] - 71s 360ms/step - loss: 0.9633 - acc: 0.6585 - val_loss: 1.1245 - val_acc: 0.6218
Epoch 6/25
196/196 [==============================] - 50s 255ms/step - loss: 0.9118 - acc: 0.6783 - val_loss: 0.8171 - val_acc: 0.7147
Epoch 7/25
196/196 [==============================] - 50s 253ms/step - loss: 0.8723 - acc: 0.6938 - val_loss: 0.8013 - val_acc: 0.7231
Epoch 8/25
196/196 [==============================] - 5

In [50]:
scores = model.evaluate(testing_images, testing_labels, verbose=1)
print("Test loss: ", scores[0])
print("Test accuracy: ", scores[1])

10000/10000 [==============================] - 2s 249us/sample - loss: 0.6284 - acc: 0.7913
Test loss:  0.6283716292381286
Test accuracy:  0.7913
